In [18]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import csv

In [11]:
df = pd.read_csv("../data/survivability/nymph/data-habitat.2020.May.28.20_08_56", error_bad_lines=False)
df.head()

b'Skipping line 8378: expected 4 fields, saw 6\nSkipping line 8379: expected 4 fields, saw 6\nSkipping line 8380: expected 4 fields, saw 6\nSkipping line 8381: expected 4 fields, saw 6\nSkipping line 8382: expected 4 fields, saw 6\nSkipping line 8383: expected 4 fields, saw 6\nSkipping line 8384: expected 4 fields, saw 6\nSkipping line 8385: expected 4 fields, saw 6\nSkipping line 8386: expected 4 fields, saw 6\nSkipping line 8387: expected 4 fields, saw 6\nSkipping line 8388: expected 4 fields, saw 6\nSkipping line 8389: expected 4 fields, saw 6\nSkipping line 8390: expected 4 fields, saw 6\nSkipping line 16061: expected 4 fields, saw 6\nSkipping line 16062: expected 4 fields, saw 6\nSkipping line 16063: expected 4 fields, saw 6\nSkipping line 16064: expected 4 fields, saw 6\nSkipping line 16065: expected 4 fields, saw 6\nSkipping line 16066: expected 4 fields, saw 6\nSkipping line 16067: expected 4 fields, saw 6\nSkipping line 16068: expected 4 fields, saw 6\nSkipping line 16069: exp

,run,tick,lifestate,name
0,1,1.0,nymph,Tick 1
1,1,1.0,nymph,Tick 0
2,1,2.0,nymph,Tick 1
3,1,2.0,nymph,Tick 0
4,1,3.0,nymph,Tick 1


In [15]:
df['run'].unique()

array([ 1,  9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 89, 97,  3, 11, 19, 27,
       35, 43, 51, 59, 67, 75, 83, 91, 99,  5, 13, 21, 29, 37, 45, 53, 61,
       69, 77, 85, 93,  4, 12, 20, 28, 36, 44, 52, 60, 68, 76, 84, 92,  6,
       14, 22, 30, 38, 46, 54, 62, 70, 78, 86, 94,  7, 15, 23, 31, 39, 47,
       55, 63, 71, 79, 87, 95,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88,
       96,  2, 10, 18, 26, 34, 42, 50, 58, 66, 74, 82, 90, 98])

In [16]:
# Get some info about the runs
for run in df.groupby('run'):
    df = run[1]
    print("Run {} made it {} days w/ {} Ixodes".format(run[0], df['tick'].max(), df['name'].value_counts().count()))

Run 1 made it 89.0 days w/ 2 Ixodes
Run 2 made it 89.0 days w/ 2 Ixodes
Run 3 made it 89.0 days w/ 2 Ixodes
Run 4 made it 89.0 days w/ 2 Ixodes
Run 5 made it 179.0 days w/ 2 Ixodes
Run 6 made it 179.0 days w/ 2 Ixodes
Run 7 made it 89.0 days w/ 2 Ixodes
Run 8 made it 266.0 days w/ 2 Ixodes
Run 9 made it 179.0 days w/ 2 Ixodes
Run 10 made it 89.0 days w/ 2 Ixodes
Run 11 made it 128.0 days w/ 2 Ixodes
Run 12 made it 179.0 days w/ 3 Ixodes
Run 13 made it 150.0 days w/ 3 Ixodes
Run 14 made it 89.0 days w/ 3 Ixodes
Run 15 made it 89.0 days w/ 3 Ixodes
Run 16 made it 89.0 days w/ 3 Ixodes
Run 17 made it 179.0 days w/ 3 Ixodes
Run 18 made it 179.0 days w/ 3 Ixodes
Run 19 made it 89.0 days w/ 3 Ixodes
Run 20 made it 179.0 days w/ 3 Ixodes
Run 21 made it 179.0 days w/ 3 Ixodes
Run 22 made it 179.0 days w/ 3 Ixodes
Run 23 made it 89.0 days w/ 4 Ixodes
Run 24 made it 89.0 days w/ 4 Ixodes
Run 25 made it 266.0 days w/ 4 Ixodes
Run 26 made it 266.0 days w/ 4 Ixodes
Run 27 made it 89.0 days w/ 4 Ixo

In [39]:
# get parameter mapping
# Create a dictionary and use try/except to add entries from the parameter file
param_file = "../data/survivability/nymph/params-May.28.20_08_56"
param_dict = {}
with open(param_file, 'r') as file:
    for line in file:
        result = line.replace('\t', ',').split(',') # result is a list
        try:
            param_dict[float(result[0])].append("Lifestate: {}\nHost Density: {}\nIxodes Count: {}\nHabitat Suitability: {}".format(
                result[2], result[4], result[6], result[8]))
        except KeyError:
            param_dict[float(result[0])] = "Lifestate: {}\nHost Density: {}\nIxodes Count: {}\nHabitat Suitability: {}".format(
                result[2], result[4], result[6], result[8])
        # param_dict = { float(result[0]): "Lifestate: {}\nHost Density: {}\nIxodes Count: {}\nHabitat Suitability: {}".format(
        #     result[2], result[4], result[6], float(result[8])) for rows in file }

for k, v in param_dict.items():
    print(k,v)

1.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: -1.0

2.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.01

3.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.02

4.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.03

5.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.04

6.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.05

7.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.06

8.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.07

9.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.08

10.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.09

11.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 2
Habitat Suitability: 0.1

12.0 Lifestate: nymph
Host Density: 1.0
Ixodes Count: 3
Habitat Suitability: -1.0

13.0 Lifestate

In [58]:
# Drop 'run' attribute so we don't plot it, get dummy variables for categoricals, and group by tick
for run in df.groupby('run'):
    params = param_dict[run[0]]
    cleaned_df = run[1].drop(["run"], axis=1)
    dummies_df = pd.get_dummies(cleaned_df, columns=['lifestate'])
    grouped_df = dummies_df.groupby('tick')

In [59]:
# Create the plots
matplotlib.use('Qt5Agg')

fig, ax = plt.subplots(figsize=(12,7))
grouped_df.sum().plot(logy=False, ax=ax)
ax.set_title("Survivability")
ax.set_xlabel("Days")
ax.set_ylabel("Lifestate count")
props = dict(facecolor='wheat', alpha=0.5)
plt.figtext(0.76,0.9,str(params), bbox=props)
plt.show(block=True)